Cómo funciona:

Construcción de rules

Para cada columna lee de stats su tipo, min/max o lista de categorías y si el conteo < total → permite nulos.

Función validate

Recorre las reglas y avisa de: • Filas nulas donde no se permiten. • Valores numéricos fuera de rango. • Categorías no listadas en “allowed”.

Reporte de errores

Devuelve un dict con mensajes por columna; útil para logs o alertas automáticas en tu pipeline.

— Si prefieres un framework ya hecho, echa un vistazo a pandera o pandas_schema, que simplifican este patrón y te dan validación tipo–schema.

Con ese bloque de código ya tienes en tus manos un validador automático que:

Extrae estadísticas de tu DataFrame (df.describe(include='all').T).

Genera un esquema de “reglas” (rangos numéricos, categorías permitidas, si acepta nulos).

Aplica esas reglas a cualquier DataFrame (el mismo u otro) y te devuelve una lista de errores por columna.

¿Qué haces con esto?

𝐂𝐨𝐦𝐩𝐫𝐨𝐛𝐚𝐫 𝐥𝐚 𝐜𝐚𝐥𝐢𝐝𝐚𝐝 𝐝𝐞 𝐭𝐮 𝐝𝐚𝐭𝐨𝐬

Tras cargar y limpiar datos, ejecuta validate(df, rules).

Si hay filas con valores fuera de rango o categorías inesperadas, te lo informa.

Así detectas errores de scraping, conversiones mal hechas o outliers.

𝐏𝐞𝐫𝐟𝐢𝐥𝐚𝐫 𝐧𝐮𝐞𝐯𝐚𝐬 𝐟𝐮𝐧𝐜𝐢𝐨𝐧𝐚𝐥𝐢𝐝𝐚𝐝𝐞𝐬

Mejora las reglas: por ejemplo, excluye el 1% de outliers ajustando min/max.

Añade validaciones de longitud de string, formatos de fecha específicos, etc.

𝐈𝐧𝐭𝐞𝐠𝐫𝐚𝐫 𝐞𝐧 𝐭𝐮 𝐩𝐢𝐩𝐞𝐥𝐢𝐧𝐞 𝐝𝐞 𝐝𝐚𝐭𝐨𝐬

Inserta este bloque al final de tu ETL o notebook.

Si validate arroja errores, detén el pipeline, corrige origen de datos o ajusta reglas.

Te permitirá automatizar chequeos y asegurar que nunca llegan datos “rotos” a tu modelo.

𝐋𝐨𝐠𝐬 𝐲 𝐦𝐞𝐭𝐫𝐢𝐜𝐚𝐬

Guarda validation_errors en un archivo o base de datos.

Monitoriza cuántos errores tienes por día/versión de dataset para medir estabilidad.

𝐔𝐬𝐚 𝐟𝐫𝐚𝐦𝐞𝐰𝐨𝐫𝐤𝐬 𝐞𝐬𝐭𝐚𝐧𝐝𝐚𝐫𝐝

Si te interesa un enfoque más declarativo, mira pandera:

Defines un esquema con Column, Range, Category, etc.

Aplicas schema.validate(df) y obtienes reportes claros y tipados.

— En resumen, lo que haces con ese código es convertir tu EDA en un guardia que vigila cada fila/columna de tus datos. Cada vez que ingrese un nuevo batch, lo pasas por validate y sólo dejas avanzar datos “limpios”. Esto eleva tu EDA de un análisis manual a un proceso reproducible y robusto.

In [ ]:
import pandas as pd

# 1. Cálculo de estadísticas
stats = df.describe(include='all').T

# 2. Generación de reglas en un dict
rules = {}

for col, row in stats.iterrows():
    n_total = df.shape[0]
    n_nonnull = int(row['count'])
    # Regla común: ¿permitimos nulos?
    allow_null = (n_nonnull < n_total)
    
    if pd.api.types.is_numeric_dtype(df[col]):
        # Para numéricos: rango [min, max]
        rules[col] = {
            'type': 'numeric',
            'min': row['min'],
            'max': row['max'],
            'allow_null': allow_null
        }
    else:
        # Para categóricos/objetos: lista de valores permitidos
        unique_vals = list(df[col].dropna().unique())
        rules[col] = {
            'type': 'categorical',
            'allowed': unique_vals,
            'allow_null': allow_null
        }

# 3. Función de validación
def validate(df, rules):
    errors = {}
    for col, rule in rules.items():
        series = df[col]
        # 3.1 Nulos no permitidos
        if not rule['allow_null']:
            n_null = series.isnull().sum()
            if n_null:
                errors.setdefault(col, []).append(f"{n_null} nulls")
        # 3.2 Chequeo por tipo
        if rule['type'] == 'numeric':
            below = (series < rule['min']).sum()
            above = (series > rule['max']).sum()
            if below or above:
                errors.setdefault(col, []).append(
                  f"{below} below {rule['min']}, {above} above {rule['max']}"
                )
        else:  # categórico
            invalid = series[~series.isin(rule['allowed']) & series.notna()]
            if not invalid.empty:
                errs = invalid.value_counts().to_dict()
                errors.setdefault(col, []).append(f"invalid categories: {errs}")
    return errors

# 4. Ejecutar validación
validation_errors = validate(df, rules)
if validation_errors:
    for col, msgs in validation_errors.items():
        print(f"{col}:")
        for m in msgs:
            print("   -", m)
else:
    print("✅ Todos los checks pasaron correctamente.")
